<a href="https://colab.research.google.com/github/esis-a-i/Generative-Models/blob/main/HWs/hw1/hw_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# МОиВС "Генеративные модели", 5-й модуль

# Homework 1

В этой домашней работе вам предстоит добавить к BERT'у декодерную часть и решить задачу генерации суммаризаций для текстов новостей на русском языке.

Дополнительно к этому на отличную оценку потребуется реализовать подсчет метрик качества и менее жадную стратегию выбора следующего токена для генерации.

*Мы сразу вас предостерегаем попасть в петлю бесконечного дообучения модели. Эта домашка не на пробитие скора. Мы будем проверять, что вы, в целом, сделали все верно и смогли получить какую-то более-менее адекватную (такую, которая заметно лучше той, что была до начала обучения) генерацию. Таким образом, если вы видите, что модель учится, не надо дообучать её сутками. Нескольких часов точно должно хватить.*



---


---
По любым вопросам касательно этой домашней работы обращайтесь ко своим ассистентам




In [ ]:
%%bash
pip install transformers datasets evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AutoTokenizer
from torch import optim
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import matplotlib.pyplot as plt

## Подготовка данных (0.5 балла)

Мы воспользуемся датасетом с 🤗 Ильи Гусева "gazeta". Он представляет собой пары (полный текст новости -- его саммари). Пары были взяты с одноименного сайта в домене .ru

Более подробно про датасет можно прочитать [здесь](https://huggingface.co/datasets/IlyaGusev/gazeta)



In [ ]:
# Загрузим данные с попощью библиотеки библиотеки datasets

from datasets import load_dataset
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0", split='train[:5%]')

Вы должны помнить, что тексты перед подачей в модель необходимо **токенизировать**.

Добавьте паддинг до `max_length=512` для обучающих данных, а также до `max_length=128` для меток.

Используйте обрезку текстов, длина которых в токенах превышает `max_length`

In [ ]:
# Подготовим данные для модели Bert

model_name = 'deepvk/bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples, use_padding=True):
  inputs = examples['text']
  targets = examples['summary']
  if use_padding:
    model_inputs = tokenizer(
          inputs,
          padding='max_length',
          max_length=512,
          truncation=True,
      )
    labels = tokenizer(
            targets,
            padding='max_length',
            max_length=128,
            truncation=True
        )
  else:
    model_inputs = tokenizer(
          inputs,
          padding=False,
          max_length=512,
          truncation=True,
      )
    labels = tokenizer(
            targets,
            padding=False,
            max_length=128,
            truncation=True
        )

  model_inputs['labels'] = labels['input_ids']

  return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess, batched=False)
tokenized_dataset.set_format('torch')

Размер батча советуем подбирать таким образом, чтоб утилизировать максимум доступной VRAM

In [ ]:
from torch.utils.data import DataLoader

batch_size = 8

train_size = int(0.7 * len(tokenized_dataset))
val_size = len(tokenized_dataset) - train_size

train_dataset, eval_dataset = torch.utils.data.random_split(tokenized_dataset, [train_size, val_size])

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size
)

eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=False,
    batch_size=batch_size
)

## Реализация Decoder-cети (3 балла)

В данном разделе вам необходимо **реализовать собственный декодер для генерации текста**.

Можете вдохновляться кодом с семинара 1 по GPT. В инициализации весов стоит (но необязательно) проявить смекалку

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

# Класс модели для суммаризации на основе BERT с кастомным декодером

class BertSummarizer(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', hidden_size=768, num_decoder_layers=3, num_heads=8, dropout=0.1):
        super(BertSummarizer, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hidden_size = hidden_size

        # Эмбеддинги для токенов на входе в декодер
        self.embedding = nn.Embedding(self.bert.config.vocab_size, hidden_size)
        if hidden_size == 768:
            self.embedding.weight = nn.Parameter(self.bert.embeddings.word_embeddings.weight.clone())

        decoder_layer = nn.TransformerDecoderLayer(d_model=hidden_size, nhead=num_heads, dropout=dropout, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.fc_out = nn.Linear(hidden_size, self.bert.config.vocab_size)


        self.softmax = nn.LogSoftmax(dim=-1)


    # Функция для создания маски для предотвращения заглядывания вперед в декодере
    def generate_square_subsequent_mask(self, T):
        return  torch.triu(torch.ones(T, T), diagonal=1).bool()


    def forward(self, input_ids, attention_mask, decoder_input_ids):
        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state  # Выходы BERT для использования в декодере

        # Эмбеддинги для входных токенов декодера
        embedded = self.embedding(decoder_input_ids)

        # Генерация маски для предотвращения заглядывания вперед
        decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(1)).to(input_ids.device)
        decoder_output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)

        output = self.fc_out(decoder_output)
        output = output
        return self.softmax(output)

    def generate(self, input_ids, attention_mask, tokenizer, max_len=50):
        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state
        batch_size = input_ids.size(0)
        # Начинаем с токена [CLS] или [BOS] (начало последовательности)

        decoder_input_ids = torch.full((batch_size, 1), tokenizer.cls_token_id, dtype=torch.long).to(input_ids.device)
        memory = memory
        generated_tokens = []

        for _ in range(max_len):
            embedded = self.embedding(decoder_input_ids)

            # Генерация маски для предотвращения заглядывания вперед
            decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(1)).to(input_ids.device)
            decoder_output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)
            output = self.fc_out(decoder_output.transpose(0, 1))

            # Получаем индекс токена с наибольшей вероятностью.
            # Помните, если EOS предсказан, прекращаем генерацию

            next_token_id = output.argmax(dim=-1)[-1, :]
            if next_token_id == tokenizer.eos_token_id:
                break
            decoder_input_ids = torch.cat([decoder_input_ids, next_token_id.unsqueeze(0)], dim=-1)

        generated_sequence = tokenizer.decode(decoder_input_ids.squeeze().tolist(), skip_special_tokens=True)

        return generated_sequence

In [ ]:
# Инициализируем нашу модель и посморим на ее архитектруру

model = BertSummarizer(bert_model_name=model_name)
model = model.to('cuda')
model

BertSummarizer(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# Посмотрим на генерацию без обучения

eval_data_sample = next(iter(eval_dataloader))
model.generate(eval_data_sample['input_ids'][:1].to('cuda'), eval_data_sample['attention_mask'][:1].to('cuda'), tokenizer)

'##ведо 4х злость философия стих быстрыи детка ##ехав ##рота ##ович ошибо ##мара ##чника 👉🏻 ##хуя стипен ##чатка финала консультацию звуки обошел космоса блокно выборы издалека ще ##лет нуле пузы упор ##нимал цени графи ##78 западе принадлежат гибели круглые брюки покинуть ##илось ##разил волок хол бре mult юбилеи украине храбро roy'

## Обучение модели (1 балл)

<small> 0.25 балла за простейший рабочий цикл; </small>

<small> +0.5 балла за графики для лосса и метрик на трейне и валидации.</small>

В данном разделе вам необходимо **реализовать цикл для обучения модели**


In [ ]:
# Пример обучения на одной итерации

def train_step(model, input_ids, attention_mask, decoder_input_ids, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask, decoder_input_ids)
    shifted_labels = torch.cat([decoder_input_ids[:, 1:].to('cpu'), torch.full((decoder_input_ids.size(0), 1), tokenizer.pad_token_id)], dim=1).to('cuda')
    loss = criterion(outputs.view(-1, outputs.size(-1)), shifted_labels.view(-1))
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
import torch.optim as optim
from tqdm import tqdm

def evaluate_step(model, input_ids, attention_mask, decoder_input_ids, criterion):
  model.eval()
  with torch.no_grad():
    outputs = model(input_ids, attention_mask, decoder_input_ids)

    loss = criterion(outputs.view(-1, outputs.size(-1)), decoder_input_ids.view(-1))
  return loss.item()


def train_loop(model, train_dataloader, eval_dataloader, epochs=3, learning_rate=1e-4):
  optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
  criterion = nn.CrossEntropyLoss()


  train_losses = []
  eval_losses = []
  for epoch in range(epochs):
      print(f"Epoch {epoch + 1}/{epochs}")
      model.train()
      epoch_train_loss = 0
      for i, batch in enumerate(tqdm(train_dataloader)):

          if epoch==1:
            for param in model.parameters():
                if param.requires_grad==True:
                    param.lr = learning_rate*(i/len(train_dataloader))

          input_ids = batch['input_ids'].to('cuda')
          attention_mask = batch['attention_mask'].to('cuda')
          decoder_input_ids = batch['labels'].to('cuda')

          loss = train_step(model, input_ids, attention_mask, decoder_input_ids, optimizer, criterion)
          epoch_train_loss += loss

      train_losses.append(epoch_train_loss / len(train_dataloader))
      print(f"Train Loss: {train_losses[-1]}")

      model.eval()
      epoch_eval_loss = 0
      for batch in tqdm(eval_dataloader):
          input_ids = batch['input_ids'].to('cuda')
          attention_mask = batch['attention_mask'].to('cuda')
          decoder_input_ids = batch['labels'].to('cuda')

          loss = evaluate_step(model, input_ids, attention_mask, decoder_input_ids, criterion)
          epoch_eval_loss += loss

      eval_losses.append(epoch_eval_loss / len(eval_dataloader))
      print(f"Eval Loss: {eval_losses[-1]}")

  return train_losses, eval_losses

In [ ]:
train_losses, eval_losses = train_loop(model, train_dataloader, eval_dataloader, epochs=3)

Epoch 1/3


100%|██████████| 267/267 [04:00<00:00,  1.11it/s]


Train Loss: 3.727054873655798


100%|██████████| 115/115 [00:30<00:00,  3.81it/s]


Eval Loss: 4.438230085372925
Epoch 2/3


...

Eval Loss: 4.562969232642132
Epoch 3/3


100%|██████████| 267/267 [04:02<00:00,  1.10it/s]


Train Loss: 2.6230416338095504


100%|██████████| 115/115 [00:29<00:00,  3.92it/s]

Eval Loss: 4.669821784807288


In [ ]:
eval_data_sample = next(iter(eval_dataloader))
model.generate(eval_data_sample['input_ids'][:1].to('cuda'), eval_data_sample['attention_mask'][:1].to('cuda'), tokenizer)

'« манчестер юнаитед » в первом матче « реал » в первом матче против « реал » в первом матче против « реал » в первом матче против « реал » и « реал » и « реал » и « реал » и « реал » и « реал'

## Метрики качества (1 балл)

<small>По 0.33 балла за реализацию каждой из предлагаемых метрик</small>

**Реализуйте функицию для подсчета метрик качества суммаризации.**

Докуметация по некотрым метрикам:
 1. [HuggingFace Rouge](https://huggingface.co/spaces/evaluate-metric/rouge)
 2. [HuggingFace Bleu](https://huggingface.co/spaces/evaluate-metric/bleu)
 3. [HuggingFace BERT Score](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [ ]:
!pip install rouge_score
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
def compute_metrics(predictions, references):
    rouge = evaluate.load('rouge')
    bleu = evaluate.load('bleu')
    bertscore = evaluate.load('bertscore')

    rouge_result = rouge.compute(predictions=predictions, references=references)
    bleu_result = bleu.compute(predictions=predictions, references=references)
    bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="ru")

    metrics = {
        "rouge1": rouge_result['rouge1'],
        "rouge2": rouge_result['rouge2'],
        "rougeL": rouge_result['rougeL'],
        "bleu": bleu_result['bleu'],
        "bert_score_f1": sum(bertscore_result['f1']) / len(bertscore_result['f1']),
    }

    return metrics

In [ ]:
def evaluation(model, eval_dataloader, tokenizer, max_len=50):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')
            labels = batch['labels']

            batch_predictions = []
            for i in range(input_ids.size(0)):
                output = model.generate(input_ids[i].unsqueeze(0), attention_mask[i].unsqueeze(0), tokenizer, max_len)
                batch_predictions.append(output)

            predictions.extend(batch_predictions)

            labels = labels.detach().cpu().numpy()
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            references.extend(decoded_labels)

    assert len(predictions) == len(references), "Количество предсказаний не совпадает с количеством ссылок"

    metrics = compute_metrics(predictions, references)
    print("Evaluation Metrics:", metrics)
    return metrics

In [ ]:
evaluation(model, eval_dataloader, tokenizer)

Evaluating: 100%|██████████| 115/115 [04:02<00:00,  2.11s/it]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Evaluation Metrics: {'rouge1': 0.0014775135373852409, 'rouge2': 0.0, 'rougeL': 0.001491086327151901, 'bleu': 0.0, 'bert_score_f1': 0.6300963460421953}


{'rouge1': 0.0014775135373852409,
 'rouge2': 0.0,
 'rougeL': 0.001491086327151901,
 'bleu': 0.0,
 'bert_score_f1': 0.6300963460421953}

## Обучение модели (0.5 балла)
**Обучите модель, сохраните лучшую версию** (метод `.save_pretrained()` объекта класса AutoModel... или `torch.save()`) **и добавьте пример генерации**. Учтите, что если изменялся токенизатор (а лучше просто по умолчанию), его тоже нужно сохранить. Если планируете продолжить обучение

Для сравнения оценки качества генерации по значениям реализованных метрик можете запустить ruT5-small без дообучения. Мы намеренно даем бейзлайн именно в таком виде.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("YOUR MODEL")
summary = #<YOUR CODE HERE>

## Реализация менее жадных стратегий выбора следующего токена (4 балла)
Всегда ли выбор наиболее вероятного токена на каждом шаге – это лучшая стратегия для генерации текста?

<details>
    <summary>Спойлер</summary>
    <p>Нет</p>
</details>

**Сравнение стратегий для генерации текста:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br><br/> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br><br/> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words <br/> that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br><br/> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps <br/> the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br><br/> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br><br/> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities <br/> by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br><br/> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br><br/> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br><br/> **Cons:** Computationally more complex and requires a good loss function. |

Ссылки на докуметацию:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

**1. Дополните метод `generate` в модели, чтобы получать топ-k самых вероятных токена и их "вероятности"** (1 балл).   

**2. Реализуйте стратегию Nucleus Sampling в методе `generate`** (1 балл)

**3. Реализуйте стратегию Beam Search** (2 балла)

Получилось ли улучшить генерацию?

In [ ]:
import torch.nn.functional as F

def top_k_sampling(logits, k):
    top_k_logits, top_k_indices = torch.topk(logits, k)
    top_k_probs = F.softmax(top_k_logits, dim=-1)
    next_token = torch.multinomial(top_k_probs, num_samples=1)
    return top_k_indices[next_token]

In [ ]:
def nucleus_sampling(logits, p):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
    sorted_indices_to_remove = cumulative_probs > p

    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    sorted_logits[sorted_indices_to_remove] = -float('Inf')

    next_token = torch.multinomial(F.softmax(sorted_logits, dim=-1), num_samples=1)
    return sorted_indices[next_token]

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

# Класс модели для суммаризации на основе BERT с кастомным декодером и выбором стратегии

class BertSummarizerCustomStrategy(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', hidden_size=768, num_decoder_layers=3, num_heads=8, dropout=0.1):
        super(BertSummarizerCustomStrategy, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hidden_size = hidden_size

        # Эмбеддинги для токенов на входе в декодер
        self.embedding = nn.Embedding(self.bert.config.vocab_size, hidden_size)

        #<YOUR CODE HERE>
        decoder_layer = nn.TransformerDecoderLayer(d_model=hidden_size, nhead=num_heads, dropout=dropout)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.fc_out = nn.Linear(hidden_size, self.bert.config.vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    # Функция для создания маски для предотвращения заглядывания вперед в декодере
    def generate_square_subsequent_mask(self, size):
        #<YOUR CODE HERE>
        mask = torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)
        return mask


    def forward(self, input_ids, attention_mask, decoder_input_ids):
        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state  # Выходы BERT для использования в декодере

        # Эмбеддинги для входных токенов декодера
        embedded = self.embedding(decoder_input_ids).transpose(0, 1)
        memory = memory.transpose(0, 1)

        decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(0)).to(input_ids.device)

        decoder_output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)

        #<YOUR CODE HERE>
        output = self.fc_out(decoder_output.transpose(0, 1))
        return output

    def generate(self, input_ids, attention_mask, tokenizer, max_len=50, strategy='top_k', top_k=5, top_p=0.9, num_beams=3):
      if strategy == 'beam_search':
        return self.beam_search(input_ids, attention_mask, tokenizer, max_len, num_beams)

      encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
      memory = encoder_outputs.last_hidden_state.transpose(0, 1)
      batch_size = input_ids.size(0)

      decoder_input_ids = torch.full((batch_size, 1), tokenizer.cls_token_id, dtype=torch.long).to(input_ids.device)

      for _ in range(max_len):
          embedded = self.embedding(decoder_input_ids).transpose(0, 1)
          decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(0)).to(input_ids.device)
          decoder_output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)
          output = self.fc_out(decoder_output.transpose(0, 1))
          next_token_logits = output[:, -1, :]

          if strategy == 'top_k':
              next_token = top_k_sampling(next_token_logits, top_k)
          elif strategy == 'nucleus':
              next_token = nucleus_sampling(next_token_logits, top_p)
          else:
              next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

          decoder_input_ids = torch.cat((decoder_input_ids, next_token), dim=1)

          if next_token.item() == tokenizer.eos_token_id:
              break

      return tokenizer.decode(decoder_input_ids.squeeze().tolist(), skip_special_tokens=True)

    def beam_search(self, input_ids, attention_mask, tokenizer, max_len=50, num_beams=3):
      encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
      memory = encoder_outputs.last_hidden_state.transpose(0, 1)
      batch_size = input_ids.size(0)

      decoder_input_ids = torch.full((batch_size * num_beams, 1), tokenizer.cls_token_id, dtype=torch.long).to(input_ids.device)
      beam_scores = torch.zeros((batch_size * num_beams, 1), dtype=torch.float, device=input_ids.device)

      for _ in range(max_len):
          embedded = self.embedding(decoder_input_ids).transpose(0, 1)
          decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(0)).to(input_ids.device)
          decoder_output = self.decoder(tgt=embedded, memory=memory.repeat(1, num_beams, 1), tgt_mask=decoder_attention_mask)
          output = self.fc_out(decoder_output.transpose(0, 1))
          next_token_logits = output[:, -1, :]

          next_token_scores = F.log_softmax(next_token_logits, dim=-1)
          scores = beam_scores + next_token_scores
          scores = scores.view(batch_size, num_beams * self.bert.config.vocab_size)

          next_beam_scores, next_beam_tokens = torch.topk(scores, num_beams, dim=1)
          beam_indices = next_beam_tokens // self.bert.config.vocab_size
          decoder_input_ids = torch.cat((decoder_input_ids[beam_indices], next_beam_tokens % self.bert.config.vocab_size), dim=2)

      generated_sequences = []
      for i in range(batch_size):
          sequence = decoder_input_ids[i * num_beams + beam_indices[i][0], :]
          generated_sequences.append(tokenizer.decode(sequence.tolist(), skip_special_tokens=True))
      return generated_sequences


In [ ]:
# Инициализируем нашу модель и посморим на ее архитектруру

model = BertSummarizerCustomStrategy(bert_model_name=model_name)
model = model.to('cuda')
model

BertSummarizerCustomStrategy(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

## Послевкусие (0 баллов)

Если эта домашняя работа показалась вам недостаточно большой, предлагаем провести следующий эксперимент:

- от имеющейся модели "откусить" только декодерную часть (откусить также можно от ruT5-small);
- немного дообучить (что называется, по вкусу);
- посмотреть качество генерации по метрикам и "глазами";
- сравнить полученное с Encoder-Decoder архитектурой;
- ответить на вопрос "Дает ли применение Encoder-Decoder архитектуры значительный буст в качестве генерации, или это некоторый overkill?" (базово, ответ лежит на поверхности 😸)

Ещё более опционально можно:
- почитать про возможности генерации Encoder-only архитектурными решениями (BERT, e.g.)
- сравнить с генерацией только Decoder'ом и both Encoder-Decoder'ом;
- в т.ч. подобрать число обучаемых параметров таким образом, чтоб оно было примерно одинаковым для каждого инстанса моделей (их, инстансов, будет 3 -- только энкодер, только декодер и энкодер-декодер).

*Вообще ориентироваться следует на следующее утверждение: "Только энкодерные архитектуры (BERT, e.g.) хороши для понимания текста (получения эмеддингов), лишь декодерные (GPT, например) -- для генерации, энкодер-декодерные (скажем, T5) -- для обеих задач"*